In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import pickle

clf = pickle.load(open("classifier.pck", "rb"))
enc = pickle.load(open("encoder.pck", "rb"))

next_matchday_df = pickle.load(open("next_matchday_df.pck", "rb"))
next_matchday_df.drop("resultClass", axis=1, inplace=True)
next_matchday_df_prep = next_matchday_df.drop(["date", "teamHomeName", "teamAwayName"], axis=1)
predictions = clf.predict(next_matchday_df_prep)
next_matchday_df["prediction"] = enc.inverse_transform(predictions)
next_matchday_df[["date", "teamHomeName", "teamAwayName", "prediction"]]

,date,teamHomeName,teamAwayName,prediction
11630,2025-05-17 15:30:00,FC Augsburg,1. FC Union Berlin,1:1
11631,2025-05-17 15:30:00,1. FSV Mainz 05,Bayer Leverkusen,1:1
11629,2025-05-17 15:30:00,SC Freiburg,Eintracht Frankfurt,2:0
11632,2025-05-17 15:30:00,Borussia Mönchengladbach,VfL Wolfsburg,1:1
11627,2025-05-17 15:30:00,TSG 1899 Hoffenheim,FC Bayern München,1:1
11626,2025-05-17 15:30:00,Borussia Dortmund,Holstein Kiel,2:1
11625,2025-05-17 15:30:00,RB Leipzig,VfB Stuttgart,2:1
11628,2025-05-17 15:30:00,1. FC Heidenheim 1846,Werder Bremen,2:1
11633,2025-05-17 15:30:00,FC St. Pauli,VfL Bochum,2:0


In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import dotenv
import os

dotenv.load_dotenv(".env")
os.getenv("EMAIL")

driver = webdriver.Chrome()
driver.get(os.getenv("LINK-TIPPABGABE"))

nav_button = driver.find_element(By.CLASS_NAME, "navtoggle")
nav_button.click()

email_input = driver.find_element(By.ID, "kennung")
email_input.send_keys(os.getenv("EMAIL"))
passwort_input = driver.find_element(By.ID, "passwort")
passwort_input.send_keys(os.getenv("PASSWORT"))

login_button = driver.find_element(By.NAME, "submitbutton")
login_button.click()

In [24]:
frame = driver.switch_to.frame(driver.find_element(By.XPATH, "/html/body/div[2]/iframe"))
driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div/div[3]/div[1]/div[2]/button").click()
driver.switch_to.default_content()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/iframe"}
  (Session info: chrome=136.0.7103.93); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104ea03bc cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x0000000104e98684 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x00000001049d5fbc cxxbridge1$string$len + 90140
3   chromedriver                        0x0000000104a1d1bc cxxbridge1$string$len + 381468
4   chromedriver                        0x0000000104a5e044 cxxbridge1$string$len + 647332
5   chromedriver                        0x0000000104a113f8 cxxbridge1$string$len + 332888
6   chromedriver                        0x0000000104e647e0 cxxbridge1$str$ptr + 2585200
7   chromedriver                        0x00000

In [25]:
from difflib import get_close_matches

driver.switch_to.default_content()
tippabgabe_tabelle = driver.find_element(By.ID, "tippabgabeSpiele")
for datarow in tippabgabe_tabelle.find_elements(By.CLASS_NAME, "datarow"):
    homeTeam = datarow.find_element(By.CLASS_NAME, "col1").get_attribute("innerHTML")
    if homeTeam != "":
        selectedTeam = get_close_matches(homeTeam, next_matchday_df["teamHomeName"])[0]
        prediction = next_matchday_df[next_matchday_df["teamHomeName"] == selectedTeam]["prediction"].iloc[0].split(":")
        inputs = datarow.find_elements(By.TAG_NAME, "input")
        inputs[1].clear()
        inputs[1].send_keys(prediction[0])
        inputs[2].clear()
        inputs[2].send_keys(prediction[1])

In [23]:
driver.find_element(By.NAME, "submitbutton").click()

ElementClickInterceptedException: Message: element click intercepted: Element <button type="submit" name="submitbutton">...</button> is not clickable at point (266, 968). Other element would receive the click: <iframe src="https://cdn.privacy-mgmt.com/index.html?hasCsp=true&amp;message_id=1230027&amp;consentUUID=9fce6fb5-e928-444d-bc0e-25fcafe6c420&amp;consent_origin=https%3A%2F%2Fcdn.privacy-mgmt.com%2Fconsent%2Ftcfv2&amp;preload_message=true&amp;concatenatedUUID=9fce6fb5-e928-444d-bc0e-25fcafe6c420~~&amp;version=v1&amp;consentLanguage=de" id="sp_message_iframe_1230027" cd_frame_id_="cf95b133a6f3fd2cd7756371f1223dba" title="SP Consent Message"></iframe>
  (Session info: chrome=136.0.7103.93)
Stacktrace:
0   chromedriver                        0x0000000104ea03bc cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x0000000104e98684 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x00000001049d5fbc cxxbridge1$string$len + 90140
3   chromedriver     

In [ ]:
driver.close()